In [1]:
import pandas
import sklearn
import sys

from pandas import Series, DataFrame
import numpy as np
import xgboost as xgb
from sklearn import cross_validation

sys.path.append("../")
from common.CommonRegression import CommonRegression

In [2]:
train_df = pandas.read_csv("data/train.csv")
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [4]:
train_df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
5,0,3,male,NaN,0,0,8.4583,Q
6,0,1,male,54.0,0,0,51.8625,S
7,0,3,male,2.0,3,1,21.0750,S
8,1,3,female,27.0,0,2,11.1333,S
9,1,2,female,14.0,1,0,30.0708,C


In [5]:
train_df = train_df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
5,0,3,male,NaN,0,0,8.4583,Q
6,0,1,male,54.0,0,0,51.8625,S
7,0,3,male,2.0,3,1,21.0750,S
8,1,3,female,27.0,0,2,11.1333,S
9,1,2,female,14.0,1,0,30.0708,C


In [6]:
train_df["Age"] = train_df["Age"].fillna(train_df["Age"].mean())

In [7]:
train_df["Embarked"] = train_df["Embarked"].fillna(train_df["Embarked"].mode()[0])

In [8]:
train_df.head()


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [9]:
train_df["Embarked"] = train_df["Embarked"].astype("category")
train_df["Embarked"] = train_df["Embarked"].cat.codes
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,2
1,1,1,female,38.0,1,0,71.2833,0
2,1,3,female,26.0,0,0,7.9250,2
3,1,1,female,35.0,1,0,53.1000,2
4,0,3,male,35.0,0,0,8.0500,2


In [10]:
train_df["Sex"] = train_df["Sex"].astype("category")
train_df["Sex"] = train_df["Sex"].cat.codes
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [11]:
training_set = train_df.drop(["Survived"], axis=1)
target_set = train_df["Survived"]

print len(training_set)
print len(target_set)

891
891


In [15]:
# K-fold cross validation
kf = cross_validation.KFold(len(training_set), n_folds=3)
for train_index, test_index in kf:
    x_train, x_test = training_set.ix[train_index], training_set.ix[test_index]
    y_train, y_test = target_set.ix[train_index], target_set.ix[test_index]
        
    commonAlgo = CommonRegression(x_train, y_train, x_test, y_test)
    commonAlgo.start()
    

In [ ]:
X_train["Embarked"] = X_train["Embarked"].cat.codes

In [ ]:
X_train["Sex"] = X_train["Sex"].astype("category")
X_train["Sex"] = X_train["Sex"].cat.codes

In [127]:
# Linear Regression
linear = LinearRegression()
linear.fit(X_train, Y_train)
linear.score(X_train, Y_train)

0.39790700664304002

In [137]:
# Random Forest
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
random_forest.score(X_train, Y_train)

0.98204264870931535

In [138]:
# Logistic Regression
logistic = LogisticRegression()
logistic.fit(X_train, Y_train)
logistic.score(X_train, Y_train)

0.80471380471380471

In [140]:
# XGBoost
params = {"objective": "binary:logistic",
          "eta": 0.1,
          "max_depth": 5,
          "min_child_weight": 3,
          "silent": 1,
          "subsample": 0.7,
          "colsample_bytree": 0.7,
          "seed": 1}
num_trees=250
gbm = xgb.train(params, xgb.DMatrix(X_train, Y_train), num_trees)

In [143]:
X_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


In [144]:
X_test["Sex"] = X_test["Sex"].astype("category")
X_test["Embarked"] = X_test["Embarked"].astype("category")

In [148]:
X_test["Sex"] = X_test["Sex"].cat.codes
X_test["Embarked"] = X_test["Embarked"].cat.codes


AttributeError: Can only use .cat accessor with a 'category' dtype

In [162]:
X_test["Age"] = X_test["Age"].fillna(X_test["Age"].mean())
X_test["Embarked"] = X_test["Embarked"].fillna(X_test["Embarked"].mode())
X_test["Fare"] = X_test["Fare"].fillna(X_test["Fare"].mean())


0

In [166]:
gbm.predict(xgb.DMatrix(X_test))

array([ 0.05533627,  0.12954813,  0.07295285,  0.25502348,  0.47378823,
        0.12903664,  0.33239749,  0.13799036,  0.87516773,  0.01132281,
        0.01454793,  0.06824632,  0.94434381,  0.03344063,  0.93604952,
        0.88247466,  0.1335765 ,  0.24153914,  0.58430076,  0.5573681 ,
        0.5627864 ,  0.34554091,  0.98747444,  0.53295821,  0.94757718,
        0.01598003,  0.99302858,  0.29135734,  0.73960537,  0.118157  ,
        0.03898055,  0.19365716,  0.4935194 ,  0.0540062 ,  0.75203675,
        0.30217811,  0.1595083 ,  0.11078369,  0.064564  ,  0.4676069 ,
        0.0857121 ,  0.51594347,  0.05983258,  0.96222246,  0.97810066,
        0.18161824,  0.19635403,  0.02609157,  0.97525793,  0.74686849,
        0.54755002,  0.16255261,  0.90353125,  0.63781577,  0.11692339,
        0.00773212,  0.02417321,  0.08175397,  0.06331989,  0.99529606,
        0.03049327,  0.42031917,  0.08353469,  0.78788394,  0.35910112,
        0.84831882,  0.80244088,  0.03751808,  0.31218386,  0.85

In [165]:
random_forest.predict(X_test)

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0,